# A first pipeline

## Example text

In [1]:
with open('example.txt', 'r') as f:
    text = f.read()

In [2]:
print(text)

Motif :
Le patient est admis le 29 août pour des difficultés respiratoires.

Antécédents familiaux :
Le père est asthmatique, sans traitement particulier.

HISTOIRE DE LA MALADIE
Le patient dit avoir de la toux depuis trois jours. Elle a empiré jusqu'à nécessiter un passage aux urgences.

Conclusion
Possible infection au coronavirus



## Defining the Spacy pipeline

In [3]:
# Importating Spacy
import spacy

In [4]:
# Loading EDS-NLP pipelines
import edsnlp.components

In [ ]:
# Creates the Spacy instance
nlp = spacy.blank('fr')

# Normalisation of accents, case and other special characters
nlp.add_pipe('normalizer')
# Detecting end of lines
nlp.add_pipe('sentences')

# Extraction of named entities
nlp.add_pipe(
    'matcher',
    config=dict(
        terms=dict(respiratoire=[
            'difficultes respiratoires',
            'asthmatique',
            'toux',
        ]),
        regex=dict(
            covid=r'(?i)(?:infection\sau\s)?(covid[\s\-]?19|corona[\s\-]?virus)',
            traitement=r'(?i)traitements?|medicaments?'),
        attr='NORM',
    ),
)

# Qualification of the entities
nlp.add_pipe('negation')
nlp.add_pipe('hypothesis')
nlp.add_pipe('family')
nlp.add_pipe('rspeech')

## Using the pipeline

In [6]:
doc = nlp(text)

In [7]:
doc

Motif :
Le patient est admis le 29 août pour des difficultés respiratoires.

Antécédents familiaux :
Le père est asthmatique, sans traitement particulier.

HISTOIRE DE LA MALADIE
Le patient dit avoir de la toux depuis trois jours. Elle a empiré jusqu'à nécessiter un passage aux urgences.

Conclusion
Possible infection au coronavirus

---

Processing by EDS-NLP (and Spacy in general) are all non-destructive :

In [8]:
# Non-destruction
doc.text == text

True

For tasks such as normalization, EDS-NLP adds attributes to tokens, without information loss:

In [9]:
# Normalisation
print(f"{'text':<15}", 'normalisation')
print(f"{'----':<15}", '-------------')
for token in doc[3:15]:
    print(f"{token.text:<15}", f"{token.norm_}")

text            normalisation
----            -------------
Le              le
patient         patient
est             est
admis           admis
le              le
29              29
août            aout
pour            pour
des             des
difficultés     difficultes
respiratoires   respiratoires
.               .


The pipeline we defined above extracted named entities using the `matcher` component.

Since we inherit from Spacy, we can use their utilities :

In [10]:
from spacy import displacy

In [11]:
displacy.render(
    doc,
    style='ent',
    options={'colors': dict(respiratoire='green', covid='orange')},
)

Let's focus on the fist entity :

In [12]:
entity = doc.ents[0]

In [13]:
entity

difficultés respiratoires

In [14]:
entity._.negated

False

We can reformat the entities to an OMOP-like format:

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame.from_records([
    dict(
        label=ent.label_,
        start_char=ent.start_char,
        end_char=ent.end_char,
        lexical_variant=ent.text,
        negation=ent._.negated,
        family=ent._.family,
        hypothesis=ent._.hypothesis,
        rspeech=ent._.reported_speech,
    )
    for ent in doc.ents
])

In [17]:
df

,label,start_char,end_char,lexical_variant,negation,family,hypothesis,rspeech
0,respiratoire,49,74,difficultés respiratoires,False,False,False,False
1,respiratoire,113,124,asthmatique,False,True,False,False
2,traitement,131,141,traitement,True,True,False,False
3,respiratoire,206,210,toux,False,False,False,True
4,covid,310,334,infection au coronavirus,False,False,True,True
